In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np, pandas as pd
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

# The Layer Class
- tf.keras.layers.Layer class is an abstract class which will be used define custom layers
- This class have both state (layer weights) and and a transformation from inputs to outputs
- Lets see how we can use Layer class to define a Dense layer with state: the variables w and b

In [4]:
class Linear(tf.keras.layers.Layer):
    def __init__(self, units=32, input_dim=32):
        super(Linear, self).__init__()
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(initial_value=w_init(shape=(input_dim, units), dtype="float32"),
                             trainable=True)
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(initial_value=b_init(shape=(units,), dtype="float32"), trainable=True)

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b
    
# You would use a layer by calling it on some tensor input(s), much like a Python function.

x = tf.ones((2,2))
linear_layer = Linear(4,2)
y = linear_layer(x)
print(y)


tf.Tensor(
[[ 0.06529395  0.02208516 -0.00957388 -0.00714765]
 [ 0.06529395  0.02208516 -0.00957388 -0.00714765]], shape=(2, 4), dtype=float32)


In [5]:
# weights w and b are automatically tracked by the layer upon being set as layer attributes:
assert linear_layer.weights == [linear_layer.w, linear_layer.b] 

In [8]:
# Instead of initializing the values and assign them to a variable we can use add_weight method as shown below
class Linear(tf.keras.layers.Layer):
    def __init__(self, units=32, input_dim=32):
        super(Linear, self).__init__()
        self.w = self.add_weight(shape=(input_dim, units), initializer = "random_normal", trainable=True)
        self.b = self.add_weight(shape=(units,), initializer = "zeros", trainable=True)

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b
    
# You would use a layer by calling it on some tensor input(s), much like a Python function.

x = tf.ones((2,2))
linear_layer = Linear(4,2)
y = linear_layer(x)
print(y)

tf.Tensor(
[[-0.0175115  -0.10254346  0.06433796  0.03940462]
 [-0.0175115  -0.10254346  0.06433796  0.03940462]], shape=(2, 4), dtype=float32)


## Layers can have non-trainable weights
- Besides trainable weights, you can add non-trainable weights to a layer as well. 
- Such weights are meant not to be taken into account during backpropagation, when you are training the layer.

Here's how to add and use a non-trainable weight:

In [10]:
class ComputeSum(tf.keras.layers.Layer):
    def __init__(self, input_dim):
        super(ComputeSum, self).__init__()
        self.total = tf.Variable(initial_value = tf.zeros((input_dim,)), trainable=False)
    def call(self, inputs):
        self.total.assign_add(tf.reduce_sum(inputs, axis=0))
        return self.total
    
x = tf.ones((2, 2))
my_sum = ComputeSum(2)
y = my_sum(x)
print(y.numpy())
y = my_sum(x)
print(y.numpy())

[2. 2.]
[4. 4.]


In [11]:
# It's part of layer.weights, but it gets categorized as a non-trainable weight:


print("weights:", len(my_sum.weights))
print("non-trainable weights:", len(my_sum.non_trainable_weights))

# It's not included in the trainable weights:
print("trainable_weights:", my_sum.trainable_weights)

weights: 1
non-trainable weights: 1
trainable_weights: []


## Best Practice:
- Our Linear layer above took an input_dimargument that was used to compute the shape of the weights w and b in __init__()
- In many cases, you may not know in advance the size of your inputs, and you would like to lazily create weights when that value becomes known, some time after instantiating the layer.
- In the Keras API, we recommend creating layer weights in the build(self, inputs_shape) method of your layer. Like this:


In [13]:
class Linear(tf.keras.layers.Layer):
    def __init__(self, units=32):
        super(Linear,self).__init__()
        self.units = units
    def build(self, input_shape):
        self.w = self.add_weight(
            shape = (input_shape[-1], self.units),
            initializer = "random_normal",
            trainable = True
        )
        self.b = self.add_weight(
            shape = (self.units,),
            initializer = "zeros",
            trainable = True
            )
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

In [14]:
# The __call__() method of your layer will automatically run build the first time it is called. 
# You now have a layer that's lazy and thus easier to use:
# At instantiation, we don't know on what inputs this is going to get called
linear_layer = Linear(32)

# The layer's weights are created dynamically the first time the layer is called
y = linear_layer(x)

In [15]:
# Implementing build() separately as shown above nicely separates creating weights only once from using weights in every call. 
# However, for some advanced custom layers, it can become impractical to separate the state creation and computation. 
# Layer implementers are allowed to defer weight creation to the first __call__(), but need to take care that later calls use the same weights. 
# In addition, since __call__() is likely to be executed for the first time inside a tf.function, any variable creation that takes place in __call__() should be wrapped in a tf.init_scope.

## Layers are recursively composable
- If you assign a Layer instance as an attribute of another Layer, the outer layer will start tracking the weights created by the inner layer.
- We recommend creating such sublayers in the __init__() method and leave it to the first __call__() to trigger building their weights

In [16]:
class MLPBlock(keras.layers.Layer):
    def __init__(self):
        super(MLPBlock, self).__init__()
        self.linear_1 = Linear(32)
        self.linear_2 = Linear(32)
        self.linear_3 = Linear(1)

    def call(self, inputs):
        x = self.linear_1(inputs)
        x = tf.nn.relu(x)
        x = self.linear_2(x)
        x = tf.nn.relu(x)
        return self.linear_3(x)


mlp = MLPBlock()
y = mlp(tf.ones(shape=(3, 64)))  # The first call to the `mlp` will create the weights
print("weights:", len(mlp.weights))
print("trainable weights:", len(mlp.trainable_weights))

weights: 6
trainable weights: 6
